In [1]:
import psycopg2
from psycopg2.extras import execute_values
import random

conn = psycopg2.connect(
    dbname='testing',
    user='postgres',
    password='password',
    host='localhost',
    port='5432'
)
cur = conn.cursor()

query_add_type = """
    INSERT INTO type (name) VALUES (%s)"""

query_add_genre = """
    INSERT INTO genre (name) VALUES (%s)"""

query_add_media = """
    INSERT INTO media(name, genre, type) VALUES (%s, %s, %s)"""

query_add_account = """
    INSERT INTO account(name, date_created, total_reviews)
    VALUES (%s, CURRENT_DATE, 0)"""

query_get_media_id = """ 
    SELECT id from media where name = %s"""

query_get_account_id = """ 
    SELECT id from account where name = %s"""

query_add_follower = """
    WITH new_follow AS (
        INSERT INTO following (account_id, follows_id, date_followed)
        VALUES (%s, %s, CURRENT_DATE)
        RETURNING account_id, follows_id
    ),
    update_following AS (
        UPDATE account
        SET total_following = total_following + 1
        WHERE id = (SELECT account_id FROM new_follow)
        RETURNING id
    )
    UPDATE account
    SET total_followers = total_followers + 1
    WHERE id = (SELECT follows_id FROM new_follow);
"""

query_add_review = """

    WITH new_review AS (
        INSERT INTO review (account_id, media_id, rating, description, date_reviewed)
        VALUES (%s, %s, %s, %s, CURRENT_DATE)
        RETURNING account_id, media_id, rating
    ),
    update_account AS (
        UPDATE account
        SET total_reviews = total_reviews + 1
        WHERE id = (SELECT account_id FROM new_review)
        RETURNING id
    )
    UPDATE media
    SET total_reviews = total_reviews + 1,
    full_average = (
        (full_average * total_reviews + (SELECT rating FROM new_review)) 
        / (total_reviews + 1)
    )
    WHERE id = (SELECT media_id FROM new_review);
"""

query_get_follows = """
    SELECT a.id, a.name, a.total_reviews, a.total_following, a.total_followers 
    FROM following f
    JOIN account a on f.account_id = a.id
    WHERE f.follows_id = %s;"""

query_get_following = """
    SELECT a.id, a.name, a.total_reviews, a.total_following, a.total_followers 
    FROM following f
    JOIN account a ON f.follows_id = a.id
    WHERE f.account_id = %s;"""

query_get_genre = """ SELECT id from genre where name = %s"""

query_get_type = """ SELECT id from type where name = %s """

query_account_reviews = """
SELECT m.name,
       r.rating, 
       r.rating - m.full_average,
       r.description, 
       g.name AS genre_name,  
       t.name AS type_name    
FROM review r
JOIN media m ON r.media_id = m.id
LEFT JOIN review orr ON orr.media_id = r.media_id
LEFT JOIN genre g ON m.genre = g.id   
LEFT JOIN type t ON m.type = t.id    
WHERE r.account_id = %s
GROUP BY r.media_id, r.rating, r.description, m.name, m.full_average, g.name, t.name;
"""

query_search_follower_reccomendations = """
SELECT m.name AS media_name,
       AVG(r.rating) AS average_rating,
       AVG(r.rating) - m.full_average,
       CASE 
           WHEN MAX(CASE WHEN r.account_id = f.account_id THEN 1 ELSE 0 END) = 1
           THEN TRUE 
           ELSE FALSE 
       END AS has_been_reviewed
FROM review r
JOIN following f ON r.account_id = f.follows_id
JOIN media m ON r.media_id = m.id
LEFT JOIN review r2 ON r2.account_id = f.account_id AND r2.media_id = m.id
WHERE f.account_id = %s
  AND m.genre = %s
  AND m.type = %s
GROUP BY m.name, m.full_average
ORDER BY average_rating DESC;
"""

query_see_recent_follower_reviews = """
SELECT t.name AS type_name,
       g.name AS genre_name,
       m.name AS media_name,
       a.name AS account_name,
       r.rating,
       r.rating - m.full_average AS rating_difference,
       r.description
FROM review r
JOIN account a ON r.account_id = a.id
JOIN media m ON r.media_id = m.id
JOIN genre g ON m.genre = g.id  
JOIN type t ON m.type = t.id   
WHERE a.id IN (
    SELECT follows_id
    FROM following
    WHERE account_id = %s
)
ORDER BY r.date_reviewed DESC  
LIMIT 20;
"""

In [2]:
cur.execute("""
    DELETE from following;
    DELETE from review;
    DELETE from account;
    DELETE from media;
    DELETE from type;
    DELETE from genre;
""")
conn.commit()

In [3]:
#Should Be Done Before
types = ['Movie', 'Tv-Show']
cur.executemany(query_add_type, [(type,) for type in types])

# Updated list of genres
genres = ['Horror', 'Fantasy', 'Action', 'Adventure', 'Romance', 'Documentary', 
          'Drama', 'Crime', 'Sci-Fi', 'Thriller', 'History', 'Animation', 'War', 'Western']

cur.executemany(query_add_genre, [(genre,) for genre in genres])

conn.commit()


In [4]:
#MOVIES SHOULD BE PIPED, WITH ACCOUNT, REVIEW



#TEMPORARY:
movies = [
    ["The Shawshank Redemption", "Drama", "Movie"],
    ["The Godfather", "Crime", "Movie"],
    ["The Dark Knight", "Action", "Movie"],
    ["Pulp Fiction", "Crime", "Movie"],
    ["Forrest Gump", "Drama", "Movie"],
    ["Inception", "Sci-Fi", "Movie"],
    ["Fight Club", "Drama", "Movie"],
    ["The Matrix", "Action", "Movie"],
    ["Goodfellas", "Crime", "Movie"],
    ["The Lord of the Rings: The Fellowship of the Ring", "Fantasy", "Movie"],
    ["Star Wars: Episode V - The Empire Strikes Back", "Fantasy", "Movie"],
    ["The Silence of the Lambs", "Thriller", "Movie"],
    ["Schindler's List", "History", "Movie"],
    ["The Green Mile", "Drama", "Movie"],
    ["Gladiator", "Action", "Movie"],
    ["Interstellar", "Sci-Fi", "Movie"],
    ["The Lion King", "Animation", "Movie"],
    ["Saving Private Ryan", "War", "Movie"],
    ["The Departed", "Crime", "Movie"],
    ["Django Unchained", "Western", "Movie"]
]

for movie in movies:
    cur.execute(query_get_genre, (movie[1],))
    genre = cur.fetchall()[0][0]

    cur.execute(query_get_type, (movie[2],))
    type = cur.fetchall()[0][0]

    cur.execute(query_add_media, (movie[0], genre, type,))

conn.commit()

In [5]:
#SAMPLE REAL ACCOUNTS (Used For Presentation)
#---------------------------------------


media_to_review = 50
max_reviews_per_account = 10

#Sample Account Names
account_names = ['Venkat Gunturi', 'Gavin Heslip', 'Hashim Jama', 
                 'Zach Reid', 'Lucas Duncan', 'John Doe', 'Jane Smith',
                 'Kylie Tan', 'Bertha Blue', 'Tiffany Stoik']
cur.executemany(query_add_account, [(account,) for account in account_names])
conn.commit()

#Sample media
cur.execute(f""" SELECT id from media ORDER BY RANDOM() LIMIT {media_to_review}""")
media_ids = cur.fetchall()

#Sample Account (ids)
account_ids = []
for name in account_names:
    cur.execute(query_get_account_id, (name,))
    account_ids.append(cur.fetchall()[0][0])

#Create Sample Followings
for id in account_ids:
    follow_count = random.randint(1, len(account_ids) - 1)
    account_ids_to_follow = random.sample(account_ids, follow_count)
    for follower_id in account_ids_to_follow:
        if follower_id == id: continue
        cur.execute(query_add_follower, (id, follower_id))
conn.commit()  

#Create Sample Reviews:
for id in media_ids:
    review_count = random.randint(1, max_reviews_per_account)
    
    account_ids_to_review = random.sample(account_ids, review_count)

    for account_id in account_ids_to_review:
        rating = random.randint(1, 10)
        cur.execute(query_add_review, (account_id, id, rating, 'ADD OR REMOVE',))

conn.commit()


In [6]:
       
#TESTING FUNCTIONS (AS IF FROM LUCAS DUNCAN PERSPECTIVE - SQL commands, with python)
cur.execute(query_get_account_id, ("Lucas Duncan",))
example_account_id = cur.fetchall()[0][0]


#Find All People Lucas Follows
cur.execute(query_get_follows, (example_account_id,))

print("FOLLOWS LIST:")
for row in cur.fetchall():
    print(row)

#Find All People Following Lucas
print("\n FOLLOWING LIST")
cur.execute(query_get_following, (example_account_id,))
for row in cur.fetchall():
    print(row)



#Find All Reviews Made By Lucas
print(" \n REVIEWS MADE")
cur.execute(query_account_reviews, (example_account_id,))
for row in cur.fetchall():
    print(row)


#Determine Reccomendations For Given Genre, Type Not Reviewed
cur.execute(query_get_genre, ("Action",))
genre = cur.fetchall()[0][0]
cur.execute(query_get_type, ("Movie",))
type = cur.fetchall()[0][0]

cur.execute(query_search_follower_reccomendations, (example_account_id,genre, type,))
print("\n RECCOMENDATIONS (of given genre, type, from followers)")
for row in cur.fetchall():
    print(row)

cur.execute(query_see_recent_follower_reviews, (example_account_id,))
print("\n RECENT ACTIVITY IN FEED")
for row in cur.fetchall():
    print(row)




FOLLOWS LIST:
(913, 'Hashim Jama', 12, 6, 6)
(916, 'John Doe', 12, 5, 3)
(920, 'Tiffany Stoik', 10, 1, 7)
(914, 'Zach Reid', 12, 8, 5)
(911, 'Venkat Gunturi', 13, 7, 4)
(912, 'Gavin Heslip', 11, 6, 3)

 FOLLOWING LIST
(919, 'Bertha Blue', 11, 3, 6)
(913, 'Hashim Jama', 12, 6, 6)
(920, 'Tiffany Stoik', 10, 1, 7)
 
 REVIEWS MADE
('The Shawshank Redemption', 10.0, 4.285714, 'ADD OR REMOVE', 'Drama', 'Movie')
('Pulp Fiction', 4.0, -1.8888888, 'ADD OR REMOVE', 'Crime', 'Movie')
('Inception', 9.0, 3.1999998, 'ADD OR REMOVE', 'Sci-Fi', 'Movie')
('Fight Club', 8.0, 1.2222223, 'ADD OR REMOVE', 'Drama', 'Movie')
('The Matrix', 8.0, 0.16666651, 'ADD OR REMOVE', 'Action', 'Movie')
("Schindler's List", 2.0, -4.125, 'ADD OR REMOVE', 'History', 'Movie')
('The Green Mile', 3.0, -3.0, 'ADD OR REMOVE', 'Drama', 'Movie')
('Gladiator', 5.0, -0.5714288, 'ADD OR REMOVE', 'Action', 'Movie')
('Interstellar', 4.0, -0.4000001, 'ADD OR REMOVE', 'Sci-Fi', 'Movie')
('The Lion King', 7.0, 2.0, 'ADD OR REMOVE', 'Ani